# Building a house (Harvard Business Review, 1963)

Several tasks must be completed in order to build our house. Each task has a duration, tasks may be worked on simultaneously, but there is also a precedence relation. Some tasks can only be started once other tasks have been completed. The following table shows each task, its duration, and the tasks that must be completed before it starts. How fast can the house be built?
![alt text](https://hbr.org/resources/images/article_assets/hbr/6309/63508_A.gif)

## Problem Data

In [1]:
# this array stores the task names (:a, :b, ..., :x)
tasks = []
for i = 'a':'x'
    push!(tasks, Symbol(i))    # string(sym) converts back to a string, i.e. string(:hello) returns "hello"
end
println("Number of tasks = ",length(tasks))
println(tasks)
# this dictionary stores the project durations
dur = [0, 4, 2, 4, 6, 1, 2, 3, 2, 4, 10, 3, 1, 2, 3, 2, 1, 1, 2, 3, 1, 2, 5, 0]
duration = Dict(zip(tasks,dur))
println("Number of durations = ",length(dur))

# this dictionary stores the projects that a given project depends on (ancestors)
pre = ( [], [:a], [:b], [:c], [:d], [:c], [:f], [:f], [:d], [:d,:g], [:i,:j,:h], [:k],
    [:l], [:l], [:l], [:e], [:p], [:c], [:o,:t], [:m,:n], [:t], [:q,:r], [:v], [:s,:u,:w])
println("Length of predecessor list = ", length(pre))
pred = Dict(zip(tasks,pre)); #Predecessor graph information
println(pred)

Number of tasks = 24
Any[:a, :b, :c, :d, :e, :f, :g, :h, :i, :j, :k, :l, :m, :n, :o, :p, :q, :r, :s, :t, :u, :v, :w, :x]
Number of durations = 24
Length of predecessor list = 24
Dict{Any, Vector}(:o => [:l], :b => [:a], :p => [:e], :n => [:l], :j => [:d, :g], :e => [:d], :c => [:b], :h => [:f], :l => [:k], :w => [:v], :x => [:s, :u, :w], :d => [:c], :k => [:i, :j, :h], :s => [:o, :t], :v => [:q, :r], :g => [:f], :u => [:t], :q => [:p], :r => [:c], :a => Any[], :f => [:c], :m => [:l], :i => [:d], :t => [:m, :n])


## Problem Model

In [2]:
using JuMP,Clp
m = Model(Clp.Optimizer)

@variable(m, tstart[tasks])

# one-line implementation of the constraints:
#@constraint(m, link[i in tasks, j in pred[i]], tstart[i] >= tstart[j] + duration[j])

for i in tasks
    for j in pred[i]
        @constraint(m, tstart[i] >= tstart[j] + duration[j])
    end
end
@constraint(m, tstart[:a] == 0)
@objective(m, Min, tstart[:x] + duration[:x])     # total duration is start time of last task + duration of last task.

optimize!(m)
println(JuMP.value.(tstart))
println("minimum duration: ", JuMP.objective_value(m))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, Any[:a, :b, :c, :d, :e, :f, :g, :h, :i, :j  …  :o, :p, :q, :r, :s, :t, :u, :v, :w, :x]
And data, a 24-element Vector{Float64}:
  0.0
  0.0
  4.0
  6.0
 10.0
  6.0
  8.0
 11.0
 12.0
 10.0
 14.0
 24.0
 28.0
 27.0
 29.0
 16.0
 18.0
 18.0
 32.0
 29.0
 33.0
 19.0
 29.0
 34.0
minimum duration: 34.0
Coin0506I Presolve 0 (-32) rows, 0 (-24) columns and 0 (-63) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 34
Coin0511I After Postsolve, objective 34, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 34 - 0 iterations time 0.022, Presolve 0.02
